In [ ]:
import jax.numpy as jnp

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

from salve import compute_safe_path, compute, U, grad_U

based on https://github.com/nicholasRenninger/cSpaceViz_Gradient_Wavefront_planners/blob/master/planners/gradient_planner.py

In [ ]:
q_start = jnp.array([1.0, 1.0])
q_goal = jnp.array([0.0, 0.0])
q_obstacles = jnp.array([[0.5, 0.5], [0.5, 0.1], [0.1, 0.5]])

q_history = compute_safe_path(
    q_start=q_start,
    q_goal=q_goal,
    q_obstacles=q_obstacles,
    # q_obstacles=jnp.array([]),
    grad_desc_rate=1e-3,
    zero_grad_tol=1e-4,
    N=1000
)

In [ ]:
# mag_grad_U_values = jnp.linalg.norm(jnp.nan_to_num(compute(grad_U, jnp.vstack(q_history), q_goal, q_obstacles)), axis=1)
U_values = compute(U, jnp.vstack(q_history), q_goal, q_obstacles)

df = pd.DataFrame(
    {"iters": jnp.array(range(0, len(q_history))),
     "q0": [q[0] for q in q_history],
     "q1": [q[1] for q in q_history],
     "U_values": U_values,}
    #  "mag_grad_U_values": mag_grad_U_values}
)
fig_3d = px.line_3d(df, x="q0", y="q1", z="U_values", height=800)
fig_3d = fig_3d.update_traces(line={"width": 6.0, "color": "red"})

In [ ]:
grid_resolution = 100
q0, q1 = jnp.mgrid[-1:1:1/(grid_resolution), -1:1:1/grid_resolution]
q_vstacked = jnp.vstack(list(zip(q0.ravel(), q1.ravel())))

U_values = compute(U, q_vstacked, q_goal, q_obstacles)
U_values = jnp.reshape(U_values, q0.shape)

fig_3d.add_trace(go.Surface(z=U_values, x=q0, y=q1))
fig_3d.show()

In [ ]:
# grid_resolution = 100
# q0, q1 = np.mgrid[0:1:1/(grid_resolution), 0:1:1/grid_resolution]
# q_vstacked = np.vstack(list(zip(q0.ravel(), q1.ravel())))

# grad_U_values = compute(grad_U, q_vstacked, q_goal, q_obstacles)
# grad_U_values = jnp.reshape(jnp.linalg.norm(grad_U_values, axis=1), q0.shape)

# fig = go.Figure(data=[go.Surface(z=grad_U_values, x=q0, y=q1)])
# fig.show()